# Google TAPAS Demo
##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imports

In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython
tf.get_logger().setLevel('ERROR')

In [ ]:
  os.system('git clone https://github.com/google-research/tapas.git')
  os.system('pip install ./tapas')
  os.system('gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip')
  os.makedirs('results/sqa/tf_examples', exist_ok=True)
  os.makedirs('results/sqa/model', exist_ok=True)
  with open('results/sqa/model/checkpoint', 'w') as f:
    f.write('model_checkpoint_path: "model.ckpt-0"')
  for suffix in ['.data-00000-of-00001', '.index', '.meta']:
    shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  max_seq_length = 512
  vocab_file = "tapas_sqa_base/vocab.txt"
  config = tf_example_utils.ClassifierConversionConfig(
      vocab_file=vocab_file,
      max_seq_length=max_seq_length,
      max_column_id=max_seq_length,
      max_row_id=max_seq_length,
      strip_column_names=False,
      add_aggregation_candidates=False,
  )
  converter = tf_example_utils.ToClassifierTensorflowExample(config)
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, query):
  table = table_data
  examples = convert_interactions_to_examples([(table, query)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(query)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", query[position])
      print(answers)
  return all_coordinates

# Predict

In [ ]:
import mysql.connector
import sql_info
import pandas as pd
cnx = mysql.connector.connect(user=sql_info.username, password=sql_info.password, host=sql_info.server, database=sql_info.database)
df = pd.read_sql("select distinct Projects.id, client, industry, technology from Projects join Clients on Projects.client_id = Clients.id join Members on Projects.id = Members.project_id", cnx)
cnx.close()

In [ ]:
df = df[df["industry"]!="n/a"]
df = df[:10]

In [ ]:
df = df.astype(str)
data = [[]]
data[0] = list(df.columns)
data.extend(df.values.tolist())

In [ ]:
results = predict(data, ["which clients have we worked with in the Major Pharmaceuticals industry?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


id,client,industry,technology
914,Meetz,Building operators,Aerified
900,Meetz,Building operators,Quo Lux
914,Meetz,Building operators,Temp
914,Meetz,Building operators,Sonsing
914,Meetz,Building operators,Home Ing
3,Bluezoom,Major Pharmaceuticals,Stringtough
952,Browsecat,Natural Gas Distribution,Subin
686,Browsecat,Natural Gas Distribution,Bitchip
493,Browsecat,Natural Gas Distribution,Andalax
966,Oyondu,Major Banks,Pannier


> which clients have we worked with in the Major Pharmaceuticals industry?
Bluezoom


In [ ]:
results = predict(data, ["which industry uses Subin?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


id,client,industry,technology
914,Meetz,Building operators,Aerified
900,Meetz,Building operators,Quo Lux
914,Meetz,Building operators,Temp
914,Meetz,Building operators,Sonsing
914,Meetz,Building operators,Home Ing
3,Bluezoom,Major Pharmaceuticals,Stringtough
952,Browsecat,Natural Gas Distribution,Subin
686,Browsecat,Natural Gas Distribution,Bitchip
493,Browsecat,Natural Gas Distribution,Andalax
966,Oyondu,Major Banks,Pannier


> which industry uses Subin?
Natural Gas Distribution


In [ ]:
results = predict(data, ["which clients have have used Andalax technology?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


id,client,industry,technology
914,Meetz,Building operators,Aerified
900,Meetz,Building operators,Quo Lux
914,Meetz,Building operators,Temp
914,Meetz,Building operators,Sonsing
914,Meetz,Building operators,Home Ing
3,Bluezoom,Major Pharmaceuticals,Stringtough
952,Browsecat,Natural Gas Distribution,Subin
686,Browsecat,Natural Gas Distribution,Bitchip
493,Browsecat,Natural Gas Distribution,Andalax
966,Oyondu,Major Banks,Pannier


> which clients have have used Andalax technology?
Browsecat


In [ ]:
results = predict(data, ["which technology is popular among Natural Gas companies?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


id,client,industry,technology
914,Meetz,Building operators,Aerified
900,Meetz,Building operators,Quo Lux
914,Meetz,Building operators,Temp
914,Meetz,Building operators,Sonsing
914,Meetz,Building operators,Home Ing
3,Bluezoom,Major Pharmaceuticals,Stringtough
952,Browsecat,Natural Gas Distribution,Subin
686,Browsecat,Natural Gas Distribution,Bitchip
493,Browsecat,Natural Gas Distribution,Andalax
966,Oyondu,Major Banks,Pannier


> which technology is popular among Natural Gas companies?
Subin, Bitchip
